<center><img src="https://keras.io/img/logo-small.png" alt="Keras logo" width="100"><br/>
This starter notebook is provided by the Keras team.</center>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 📚 | Import Libraries 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
#Avoid Plotly issues
import plotly.io as pio
pio.renderers.default = 'notebook'


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os

/home/wei516/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

2024-12-05 13:27:13.806701: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 13:27:13.820099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733405233.838493 3387733 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733405233.843921 3387733 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting t

# 📁 | Dataset Path 

In [3]:
BASE_PATH = '/kaggle/input/llm-classification-finetuning'

In [4]:
BASE_PATH = '.'

# 📖 | Meta Data 

The competition dataset comprises user interactions from the ChatBot Arena. In each interaction, a judge presents one or more prompts to two different large language models and then indicates which model provided the more satisfactory response. The training data contains `55,000` rows, with an expected `25,000` rows in the test set.

## Files

### `train.csv`
- `id`: Unique identifier for each row.
- `model_[a/b]`: Model identity, present in train.csv but not in test.csv.
- `prompt`: Input prompt given to both models.
- `response_[a/b]`: Model_[a/b]'s response to the prompt.
- `winner_model_[a/b/tie]`: Binary columns indicating the judge's selection (ground truth target).

### `test.csv`
- `id`: Unique identifier for each row.
- `prompt`: Input prompt given to both models.
- `response_[a/b]`: Model_[a/b]'s response to the prompt.

> Note that each interaction may have multiple prompts and responses, but this notebook will use only **one prompt per interaction**. You can choose to use all prompts and responses. Additionally, prompts and responses in the dataframe are provided as string-formatted lists, so they need to be converted to literal lists using `eval()`.


## Train Data

In [5]:
df_train = pd.read_csv(f'{BASE_PATH}/train.csv', encoding='utf-8', encoding_errors='replace')
df_test = pd.read_csv(f'{BASE_PATH}/test.csv', encoding='utf-8', encoding_errors='replace')


In [6]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'notebook'

# 假設 train 是你的 DataFrame
# 整理資料：將每個欄位的 value_counts 結果整理成一個 DataFrame
data_a = df_train["winner_model_a"].value_counts().reset_index()
data_a.columns = ['Battle Outcome', 'Count']
data_a['Category'] = 'Model A'

data_b = df_train["winner_model_b"].value_counts().reset_index()
data_b.columns = ['Battle Outcome', 'Count']
data_b['Category'] = 'Model B'

data_tie = df_train["winner_tie"].value_counts().reset_index()
data_tie.columns = ['Battle Outcome', 'Count']
data_tie['Category'] = 'Tie'

# 合併所有資料
combined_data = pd.concat([data_a, data_b, data_tie], ignore_index=True)

# 過濾掉 Battle Outcome 為 0 的數據
combined_data = combined_data[combined_data['Battle Outcome'] != 0]
"""
# 繪製橫向分組柱狀圖
fig = px.bar(combined_data, y='Battle Outcome', x='Count', color='Category',
             barmode='group', orientation='h', title="Counts of Battle Outcomes for A, B, and Tie",
             height=600, text_auto=True)

# 更新圖表外觀
fig.update_layout(yaxis_title="Battle Outcome",
                  xaxis_title="Count",
                  legend_title="Category")

# 顯示圖表
fig
"""

'\n# 繪製橫向分組柱狀圖\nfig = px.bar(combined_data, y=\'Battle Outcome\', x=\'Count\', color=\'Category\',\n             barmode=\'group\', orientation=\'h\', title="Counts of Battle Outcomes for A, B, and Tie",\n             height=600, text_auto=True)\n\n# 更新圖表外觀\nfig.update_layout(yaxis_title="Battle Outcome",\n                  xaxis_title="Count",\n                  legend_title="Category")\n\n# 顯示圖表\nfig\n'

## Models


In [7]:
import plotly.io as pio
pio.renderers.default = 'notebook'

model_df = pd.concat([df_train['model_a'], df_train['model_b']])
counts = model_df.value_counts().reset_index()
counts.columns = ['LLM', 'count']
"""
fig = px.bar(counts,
             x='LLM',
             y='count',
             title='Counts of LLMs in the Training Data',
             labels={'LLM': 'LLM', 'count': 'Count'},
             color='count',
             color_continuous_scale='viridis')

fig.update_layout(xaxis_tickangle=-45)
fig.show()
"""

"\nfig = px.bar(counts,\n             x='LLM',\n             y='count',\n             title='Counts of LLMs in the Training Data',\n             labels={'LLM': 'LLM', 'count': 'Count'},\n             color='count',\n             color_continuous_scale='viridis')\n\nfig.update_layout(xaxis_tickangle=-45)\nfig.show()\n"

# Data Preprocessing

In [8]:

# Take the first prompt and its associated response
df_train["prompt"] = df_train.prompt.map(lambda x: eval(x)[0])
df_train["response_a"] = df_train.response_a.map(lambda x: eval(x.replace("null","''"))[0])
df_train["response_b"] = df_train.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# Label conversion
df_train["class_name"] = df_train[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)
df_train["class_label"] = df_train["class_name"].map({"winner_model_a": 0, "winner_model_b": 1, "winner_tie": 2})

df_train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,class_name,class_label
0,30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0,winner_model_a,0
1,53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0,winner_model_b,1
2,65089,gpt-3.5-turbo-0613,mistral-medium,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,0,0,1,winner_tie,2
3,96401,llama-2-13b-chat,mistral-7b-instruct,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,1,0,0,winner_model_a,0
4,198779,koala-13b,gpt-3.5-turbo-0314,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0,1,0,winner_model_b,1


In [9]:
# 新增一個欄位 pairs，list存放兩組 prompt-response pair(string)

def make_pairs(row):
    row["encode_fail"] = False
    try:
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except:
        prompt = ""
        row["encode_fail"] = True

    try:
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except:
        response_a = ""
        row["encode_fail"] = True

    try:
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except:
        response_b = ""
        row["encode_fail"] = True
        
    row['pairs'] = [
    f"[PROMPT] {prompt} [SEP] [RESPONSE_A] {response_a}",  # Response from Model A
    f"[PROMPT] {prompt} [SEP] [RESPONSE_B] {response_b}"  # Response from Model B
]
    return row

df_train = df_train.apply(make_pairs, axis=1)
df_test = df_test.apply(make_pairs, axis=1)

In [10]:
df_test.head()

,id,prompt,response_a,response_b,encode_fail,pairs
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran...",False,"[[PROMPT] [""I have three oranges today, I ate ..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ...",False,"[[PROMPT] [""You are a mediator in a heated pol..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p...",False,"[[PROMPT] [""How to initialize the classificati..."


In [11]:
# !pip install datasets transformers

from datasets import Dataset

#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

from transformers import AutoTokenizer

# 加载 multilingual-e5-large 的分词器
tokenizer = AutoTokenizer.from_pretrained('FacebookAI/roberta-base')

#tokenizer = BertTokenizer.from_pretrained('/kaggle/input/bert-base-uncased/pytorch/default/1/bert-base-uncased')

def preprocess_function(examples):
    # 確保 pairs 的每個元素分別處理
    encoding_a = tokenizer(
        [pair[0] for pair in examples['pairs']],  # 取 pairs 中的第一個字串
        truncation=True,
        padding='max_length',
        max_length=512
    )
    encoding_b = tokenizer(
        [pair[1] for pair in examples['pairs']],  # 取 pairs 中的第二個字串
        truncation=True,
        padding='max_length',
        max_length=512
    )

    return {
        'input_ids_a': encoding_a['input_ids'],
        'attention_mask_a': encoding_a['attention_mask'],
        'input_ids_b': encoding_b['input_ids'],
        'attention_mask_b': encoding_b['attention_mask'],
        'labels': examples['class_label']
    }

dataset = Dataset.from_pandas(df_train[['pairs','class_label']])
tokenized_dataset = dataset.map(preprocess_function, batched=True)



Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 57477/57477 [00:38<00:00, 1498.70 examples/s]


# 🍽️ | Preprocessing



In [12]:
def preprocess_function_test(examples):
    # 確保 pairs 的每個元素分別處理
    encoding_a = tokenizer(
        [pair[0] for pair in examples['pairs']],  # 取 pairs 中的第一個字串
        truncation=True,
        padding='max_length',
        max_length=512
    )
    encoding_b = tokenizer(
        [pair[1] for pair in examples['pairs']],  # 取 pairs 中的第二個字串
        truncation=True,
        padding='max_length',
        max_length=512
    )

    return {
        'input_ids_a': encoding_a['input_ids'],
        'attention_mask_a': encoding_a['attention_mask'],
        'input_ids_b': encoding_b['input_ids'],
        'attention_mask_b': encoding_b['attention_mask'],
    }

dataset_test = Dataset.from_pandas(df_test[['pairs']])
tokenized_dataset_test = dataset_test.map(preprocess_function_test, batched=1)

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 145.15 examples/s]


In [13]:
tokenized_dataset_test

Dataset({
    features: ['pairs', 'input_ids_a', 'attention_mask_a', 'input_ids_b', 'attention_mask_b'],
    num_rows: 3
})

In [14]:
# 設定張量格式
tokenized_dataset.set_format(type='torch', columns=['input_ids_a', 'attention_mask_a', 'input_ids_b', 'attention_mask_b', 'labels'])

In [15]:
tokenized_dataset_test.set_format(type='torch', columns=['input_ids_a', 'attention_mask_a', 'input_ids_b', 'attention_mask_b'])

In [16]:
# 創建 DataLoader
batch_size = 8
train_dataloader = torch.utils.data.DataLoader(tokenized_dataset, batch_size=batch_size,shuffle=True)

In [17]:
test_dataloader = torch.utils.data.DataLoader(tokenized_dataset_test, batch_size=1,shuffle=False)

# Model Structure



In [18]:
class BertComparisonModel(nn.Module):
    def __init__(self, pretrained_model_name='bert-base-uncased'):
        super(BertComparisonModel, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.pooler = nn.Linear(768 * 2, 768)
        self.classifier = nn.Linear(768, 3)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, input_ids_a, attention_mask_a, input_ids_b, attention_mask_b):
        # 分別通過 BERT 模型
        output_a = self.bert(input_ids=input_ids_a, attention_mask=attention_mask_a)
        output_b = self.bert(input_ids=input_ids_b, attention_mask=attention_mask_b)

        # 獲取 [CLS] token 的輸出作為句子嵌入
        embedding_a = output_a.last_hidden_state[:, 0, :]  # [batch_size, 768]
        embedding_b = output_b.last_hidden_state[:, 0, :]  # [batch_size, 768]

        # 將兩個句子的嵌入串接起來
        combined_embedding = torch.cat([embedding_a, embedding_b], dim=1) # [batch_size, 768 * 2]
        pooled_embedding = torch.tanh(self.pooler(combined_embedding)) # [batch_size, 768]

        # 通過分類器獲取最終的預測結果
        logits = self.classifier(pooled_embedding) # [batch_size, 3]
        probs = self.softmax(logits)

        return logits, probs
    
    def predict(self, input_ids_a, attention_mask_a, input_ids_b, attention_mask_b):
        logits, probs = self.forward(input_ids_a, attention_mask_a, input_ids_b, attention_mask_b)
        return probs



In [19]:
from transformers import AutoModel
import torch
import torch.nn as nn

class MultilingualE5ComparisonModel(nn.Module):
    def __init__(self, pretrained_model_name='intfloat/multilingual-e5-small'):
        super(MultilingualE5ComparisonModel, self).__init__()
        # 使用 AutoModel 加载预训练模型
        self.model = AutoModel.from_pretrained(pretrained_model_name)
        
        # 动态获取模型的 hidden size
        hidden_size = self.model.config.hidden_size
        
        # 根据模型的 hidden size 创建池化和分类层
        self.pooler = nn.Linear(hidden_size * 2, hidden_size)
        self.classifier = nn.Linear(hidden_size, 3)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, input_ids_a, attention_mask_a, input_ids_b, attention_mask_b):
        # 分别通过模型
        output_a = self.model(input_ids=input_ids_a, attention_mask=attention_mask_a)
        output_b = self.model(input_ids=input_ids_b, attention_mask=attention_mask_b)
        
        # 获取 [CLS] token 的输出作为句子嵌入
        embedding_a = output_a.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
        embedding_b = output_b.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
        
        # 将两个句子的嵌入串接起来
        combined_embedding = torch.cat([embedding_a, embedding_b], dim=1)  # [batch_size, hidden_size * 2]
        pooled_embedding = torch.tanh(self.pooler(combined_embedding))  # [batch_size, hidden_size]
        
        # 通过分类器获取最终的预测结果
        logits = self.classifier(pooled_embedding)  # [batch_size, 3]
        probs = self.softmax(logits)
        return logits, probs
    
    def predict(self, input_ids_a, attention_mask_a, input_ids_b, attention_mask_b):
        with torch.no_grad():
            logits, probs = self.forward(input_ids_a, attention_mask_a, input_ids_b, attention_mask_b)
            return probs

In [22]:
from transformers import AutoModel
import torch
import torch.nn as nn

class ROBERTAComparisonModel(nn.Module):
    def __init__(self, pretrained_model_name='FacebookAI/roberta-base'):
        super(ROBERTAComparisonModel, self).__init__()
        # 使用 AutoModel 加载预训练模型
        self.model = AutoModel.from_pretrained(pretrained_model_name)
        
        # 动态获取模型的 hidden size
        hidden_size = self.model.config.hidden_size
        
        # 根据模型的 hidden size 创建池化和分类层
        self.pooler = nn.Linear(hidden_size * 2, hidden_size)
        self.classifier = nn.Linear(hidden_size, 3)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, input_ids_a, attention_mask_a, input_ids_b, attention_mask_b):
        # 分别通过模型
        output_a = self.model(input_ids=input_ids_a, attention_mask=attention_mask_a)
        output_b = self.model(input_ids=input_ids_b, attention_mask=attention_mask_b)
        
        # 获取 [CLS] token 的输出作为句子嵌入
        embedding_a = output_a.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
        embedding_b = output_b.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
        
        # 将两个句子的嵌入串接起来
        combined_embedding = torch.cat([embedding_a, embedding_b], dim=1)  # [batch_size, hidden_size * 2]
        pooled_embedding = torch.tanh(self.pooler(combined_embedding))  # [batch_size, hidden_size]
        
        # 通过分类器获取最终的预测结果
        logits = self.classifier(pooled_embedding)  # [batch_size, 3]
        probs = self.softmax(logits)
        return logits, probs
    
    def predict(self, input_ids_a, attention_mask_a, input_ids_b, attention_mask_b):
        with torch.no_grad():
            logits, probs = self.forward(input_ids_a, attention_mask_a, input_ids_b, attention_mask_b)
            return probs

## Training Loop


In [23]:
model = ROBERTAComparisonModel(pretrained_model_name='FacebookAI/roberta-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=5e-5)

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from torch import nn, optim
from tqdm import tqdm

# 定義損失函數
loss_fn = nn.CrossEntropyLoss()

# 訓練設定
epochs = 3

# 假設 train_dataloader 已經定義
for epoch in range(epochs):
    model.train()
    total_loss = 0

    # 使用 tqdm 包裝 DataLoader
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", unit="batch")

    for batch in progress_bar:
        input_ids_a = batch['input_ids_a'].to(device)
        attention_mask_a = batch['attention_mask_a'].to(device)
        input_ids_b = batch['input_ids_b'].to(device)
        attention_mask_b = batch['attention_mask_b'].to(device)
        labels = batch['labels'].to(device)

        # 清空梯度
        optimizer.zero_grad()

        # 前向傳播
        logits, probs = model(input_ids_a, attention_mask_a, input_ids_b, attention_mask_b)

        # 計算損失
        loss = loss_fn(logits, labels)
        total_loss += loss.item()

        # 更新進度條的描述
        progress_bar.set_postfix(loss=loss.item())

        # 反向傳播與更新參數
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 7185/7185 [46:41<00:00,  2.56batch/s, loss=1.11]


Epoch 1, Average Loss: 1.1018


Epoch 2:  29%|██████████████████████████▊                                                                 | 2092/7185 [13:36<32:57,  2.58batch/s, loss=1.07]

In [ ]:
torch.save(model.state_dict(), "best_model.weights.pt")

# 🧪 | Prediction

In [ ]:

model = ROBERTAComparisonModel(pretrained_model_name='FacebookAI/roberta-base')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.load_state_dict(torch.load("best_model.weights.pt"))

model.eval()

In [ ]:
# Prediction loop
model.eval()  # Set model to evaluation mode
test_preds = []  # To store predictions

with torch.no_grad():
    for batch in test_dataloader:
        input_ids_a = batch['input_ids_a'].to(device)
        attention_mask_a = batch['attention_mask_a'].to(device)
        input_ids_b = batch['input_ids_b'].to(device)
        attention_mask_b = batch['attention_mask_b'].to(device)

        # Forward pass
        logits, probs = model(
            input_ids_a=input_ids_a,
            attention_mask_a=attention_mask_a,
            input_ids_b=input_ids_b,
            attention_mask_b=attention_mask_b
        )
        print(probs)
        # Predicted class
        preds = probs
        test_preds.extend(preds.cpu().numpy())  # Collect predictions


In [ ]:
test_preds

# 📬 | Submission

Following code will prepare the submission file.

In [ ]:
import numpy as np
import pandas as pd

# Define the class names directly
label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}
class_names = list(label2name.values())

# Convert predictions to a list
test_preds = np.array(test_preds).tolist()

# Assuming `df_test` has an "id" column
df_test = pd.DataFrame({'id': [1, 2, 3]})  # Example DataFrame for demonstration

# Save results
sub_df = df_test[["id"]].copy()
sub_df["winner_model_a"] = [pred[0] for pred in test_preds]
sub_df["winner_model_b"] = [pred[1] for pred in test_preds]
sub_df["winner_tie"] = [pred[2] for pred in test_preds]
sub_df.to_csv("submission.csv", index=False)  # Save to CSV

# Display the first few rows of the saved DataFrame
print(sub_df.head())

# 🔭 | Future Directions

In this notebook, we've achieved a good score with a small model and modest token length. But there's plenty of room to improve. Here's how:

1. Try bigger models like `Deberta-Base` or `Deberta-Small`, or even LLMs like `Gemma`.
2. Increase max token length to reduce loss of data.
3. Use a five-fold cross-validation and ensemble to make the model robust and get better scores.
4. Add augmentation like shuffling response orders for more robust performance.
5. Train for more epochs.
6. Tune the learning rate scheduler.

# 📌 | Reference

* [LLM Science Exam: KerasCore + KerasNLP [TPU]](https://www.kaggle.com/code/awsaf49/llm-science-exam-kerascore-kerasnlp-tpu)
* [AES 2.0: KerasNLP Starter](https://www.kaggle.com/code/awsaf49/aes-2-0-kerasnlp-starter)